In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/news-processed-dataset/keywords.pickle
/kaggle/input/news-processed-dataset/full_data.csv


In [113]:
from collections import Counter
import pandas as pd

results = Counter()

data = pd.read_csv("/kaggle/input/rambler-dataset/file.csv", sep=";", names=["Title", "Description", "Article Text", "Keywords"])
data.Keywords.apply(lambda x: list(map(lambda y: y[1:-1].lower(), x[1:-1].split(", ")))).apply(results.update)
data = pd.read_csv("/kaggle/input/russian-news-2020/news.csv")
data.tags.dropna().str.lower().str.split(", ").apply(results.update)
for sheet_name in range(4):
    data = pd.read_excel("/kaggle/input/vk-mail-dataset/data.mail.xlsx", sheet_name = sheet_name)
    data.Keywords.str.lower().str.split(", ").apply(results.update)

if "" in results:
    del results[""]

print("Total keywords:", len(results))
print("Total rows with defined keywords:", sum(results.values()))
print("Total keywords with count >= 30:", len([keyword for keyword, count in results.items() if count >= 30]))
print("Total keywords with count >= 50:", len([keyword for keyword, count in results.items() if count >= 50]))
print("Total keywords with count >= 80:", len([keyword for keyword, count in results.items() if count >= 80]))

keyword_2_id = dict()
id_2_keyword = []
for i, word in enumerate([keyword for keyword, count in results.items() if count >= 80]):
    keyword_2_id[word] = i
    id_2_keyword.append(word)

import pickle
with open('/kaggle/working/keywords.pickle', 'wb') as handle:
    pickle.dump((keyword_2_id, id_2_keyword), handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/kaggle/working/keywords.pickle', 'rb') as handle:
    keyword_2_id_, id_2_keyword_ = pickle.load(handle)
    assert keyword_2_id_ == keyword_2_id and id_2_keyword_ == id_2_keyword

Total keywords: 8281
Total rows with defined keywords: 158220
Total keywords with count >= 30: 385
Total keywords with count >= 50: 257
Total keywords with count >= 80: 190


In [135]:
sorted(results.values())[-10:]

[2913, 3147, 3295, 3601, 3736, 4015, 6266, 7547, 7728, 19825]

In [137]:
print([keyword for keyword, count in results.items() if count >= 1000])

['экономика', 'в мире', 'москва', 'общество', 'россия', 'происшествия', 'сша', 'lenta.ru', 'распространение нового коронавируса', 'коронавирус в россии', 'коронавирус covid-19', 'культура', 'туризм', 'питание', 'семья', 'статья', 'новость', 'газета.ру', 'образ жизни', 'наука и практика', 'общество и государство', 'город', 'интерьеры', 'дети 3-7 лет']


## Keyword labeling (loading)

In [6]:
import pickle

def get_keyword_vector(keywords):
    keyword_vector = [0.]*len(keyword_2_id)
    for keyword in keywords:
        id_ = keyword_2_id.get(keyword)
        if id_ is not None:
            keyword_vector[id_] = 1.
    return keyword_vector

keyword_2_id, id_2_keyword = [], []
with open('/kaggle/input/news-processed-dataset/keywords.pickle', 'rb') as handle:
    keyword_2_id, id_2_keyword = pickle.load(handle)

## Model RuBert Loading

In [7]:
!pip install gdown
!gdown http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
!tar -xzf rubert_cased_L-12_H-768_A-12_pt.tar.gz
#!pip install transformers

import numpy as np
import pandas as pd
import torch
import os
import json

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from torch.utils.data import Dataset, DataLoader
from torch import nn
from transformers import BertModel, BertTokenizer

Downloading...
From: http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz
To: /kaggle/working/rubert_cased_L-12_H-768_A-12_pt.tar.gz
100%|████████████████████████████████████████| 662M/662M [00:32<00:00, 20.3MB/s]


In [8]:
with open("rubert_cased_L-12_H-768_A-12_pt/bert_config.json", "r") as read_file, open("rubert_cased_L-12_H-768_A-12_pt/config.json", "w") as conf:
    file = json.load(read_file)
    conf.write(json.dumps(file))
!rm rubert_cased_L-12_H-768_A-12_pt/bert_config.json

In [9]:
tokenizer = BertTokenizer.from_pretrained('rubert_cased_L-12_H-768_A-12_pt')
model = BertModel.from_pretrained('rubert_cased_L-12_H-768_A-12_pt', output_hidden_states = True)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output['last_hidden_state']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Combining dataset (from three relevant to keywords)

In [123]:
full_data = pd.read_csv("/kaggle/input/rambler-dataset/file.csv", sep=";", names=["Title", "Description", "Article Text", "Keywords"])
full_data = full_data[['Title', 'Description', 'Article Text', 'Keywords']]
full_data["Keywords"] = full_data.Keywords.apply(lambda x: list(map(lambda y: y[1:-1].lower(), x[1:-1].split(", ")))).apply(", ".join)
df = pd.read_csv("/kaggle/input/russian-news-2020/news.csv")
df = df[['title', 'text', 'tags']]
df.columns = ['Title', 'Article Text', 'Keywords']
df["Description"] = ""
df["Keywords"] = data.Keywords.dropna().str.lower().str.split(", ").apply(", ".join)
full_data = pd.concat([full_data,df], axis=0, ignore_index = True)
for sheet_name in range(4):
    data = pd.read_excel("/kaggle/input/vk-mail-dataset/data.mail.xlsx", sheet_name = sheet_name)
    data = data[['Title', 'Description', 'Article Text', 'Keywords']]
    data.Keywords = data.Keywords.str.lower().str.split(", ").apply(", ".join)
    full_data = pd.concat([full_data,data], axis=0, ignore_index = True)

In [126]:
full_data.to_csv("/kaggle/working/full_data.csv")

## Dataset

In [127]:
class KeywordsDataset(Dataset):
    def __init__(self, path, tokenizer=None, only_keywords=True, only_title=False):
        self.data = full_data
        # Title,Article Text,Description,Keywords
        self.only_keywords = only_keywords
        if only_keywords:
            self.data = self.data[self.data['Keywords'].notna()]
            self.data = self.data[self.data['Description'].notna()]
            self.data = self.data[self.data['Title'].notna()]
            self.keywords = self.data.Keywords.str.lower().str.split(", ")
            self.keywords = np.array([get_keyword_vector(self.keywords.iloc[index]) for index in range(len(self.keywords))])
            indices = [i for i, k in enumerate(self.keywords) if sum(k) > 0]
            self.data.reset_index(inplace=True)
            self.data = self.data.loc[indices, :]
            self.data = self.data.loc[:, ["Title", "Article Text", "Description", "Keywords"]]
            self.keywords = self.keywords[indices]        
        if only_title:
            self.data = self.data[self.data['Class'].notna()]
        self.data.reset_index(inplace=True)
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.data.shape[0]

    def tokenize(self, text):
        if self.tokenizer is None:
            return {"text" : text}
        return self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=100)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        united_text = "; ".join([row.Title, row.Description, ""]) #row["Article Text"][:50]])
        items = {k: v.reshape(-1) for k, v in self.tokenize(united_text).items()}
        if self.only_keywords:
            items["keywords"] = torch.tensor(self.keywords[index])
        return items

In [128]:
path = "/kaggle/input/news-processed-dataset/full_data.csv"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch_size = 64
learning_rate = 1e-3
num_epochs = 10
num_workers = 2 if device.type == "cuda" else None
number_keywords = len(keyword_2_id)
embedding_dim = 768

full_dataset = KeywordsDataset(path, tokenizer=tokenizer, only_keywords=True)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, eval_dataloader = torch.utils.data.random_split(full_dataset, [train_size, test_size])
train_dataloader, eval_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers), DataLoader(eval_dataloader, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [130]:
class KeywordClassifier(nn.Module):
    def __init__(self, embedding_dim, num_classes):
        super(KeywordClassifier, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.activation = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, embeddings):
        x = self.fc1(embeddings)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        logits = self.fc3(x)
        return self.sigmoid(logits)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()
classifier_model = KeywordClassifier(embedding_dim, number_keywords).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(classifier_model.parameters(), lr=1e-4)

In [ ]:
def get_predicted_keywords(logits, id_2_keyword, top_keywords=10):
    top_k_values, top_k_indices = logits.topk(top_keywords, dim=1)
    predicted_keywords = []
    for i in range(logits.shape[0]):
        predicted_keywords.append([id_2_keyword[idx.item()] for idx in top_k_indices[i]])
    return predicted_keywords

In [ ]:
def predict_keywords(title, description, text, top_keywords=10):
    model.eval()
    classifier_model.eval()
    with torch.no_grad():
        text = "; ".join([title, description, ""])
        tokenized = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=100)
        items = {k: v.reshape(1, -1).to(device) for k, v in tokenized.items()}
        outputs = model(**items)
        embeddings = mean_pooling(outputs, items['attention_mask'])
        # print("embeddings: ", embeddings[0][:10])
        logits = classifier_model(embeddings).cpu() # expit
    # print("Logits", logits[0][:10])
    return get_predicted_keywords(logits, id_2_keyword, top_keywords)[0]

In [147]:
loss_history = []

for epoch in range(num_epochs):
    batch_losses = []
    for n_batch, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        keywords = batch.pop("keywords", None)
        with torch.no_grad():
            outputs = model(**batch)
            embeddings = mean_pooling(outputs, batch['attention_mask'])
        logits = classifier_model(embeddings)
        loss = loss_fn(logits, keywords.float())
        batch_losses.append(loss.item())
        if n_batch%40 == 0:
            accuracy = (logits.argmax(dim=1) == keywords.argmax(dim=1)).float().mean()
            print(f"Batch: {n_batch}, Loss: {loss.item()}, Accuracy: {accuracy.item()}") 
        
        if n_batch%100 == 0:
            for i_row in [10, 16]:
                row = full_dataset.data.iloc[i_row]
                print("Text: ", row.Title, row.Description)
                res = predict_keywords(row.Title, row.Description, "", top_keywords=4)
                print("Predicted:", ", ".join(res))
                print("-----------")
        
        loss.backward()
        optimizer.step()

    torch.save({
        'epoch': epoch,
        'model_state_dict': classifier_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss_history': loss_history,
    }, f"/kaggle/working/checkpoint_{epoch}.pt")

    loss_history.append(np.mean(batch_losses))
    print(f"Epoch: {epoch}, Loss: {loss_history[-1]}") #, Accuracy: {accuracy.item()}")

with torch.no_grad():
    for n_batch, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        keywords = batch.pop("keywords", None)
        outputs = model(**batch)
        embeddings = mean_pooling(outputs, batch['attention_mask'])
        logits = classifier_model(embeddings)
        loss = loss_fn(logits, keywords)
        accuracy = (logits.argmax(dim=1) == keywords.argmax(dim=1)).float().mean()
        print(f"Loss: {loss.item()}, Accuracy: {accuracy.item()}")

  0%|          | 1/1110 [00:00<11:09,  1.66it/s]

Batch: 0, Loss: 0.6944670081138611, Accuracy: 0.0
Text:  Почему Сан-Франциско оставляют богатые и захватывают бездомные Илон Маск сравнивает его пустеющий центр со сценами постапокалипсиса.
Predicted: православие и мир, статья, в мире, коронавирус в россии
-----------
Text:  В Ингушетии возрождается древняя техника ковроваляния Ее представят на форуме-выставке «Россия» на ВДНХ.
Predicted: коронавирус в россии, саудовская аравия, новости - туризм, православие и мир
-----------


  4%|▎         | 41/1110 [00:14<06:23,  2.79it/s]

Batch: 40, Loss: 0.3066663444042206, Accuracy: 0.0


  7%|▋         | 81/1110 [00:29<06:11,  2.77it/s]

...

Batch: 400, Loss: 1.0444079637527466, Accuracy: 0.203125
Text:  Почему Сан-Франциско оставляют богатые и захватывают бездомные Илон Маск сравнивает его пустеющий центр со сценами постапокалипсиса.
Predicted: новость, статья, гибель иранского генерала касема сулеймани, конституция рф
-----------
Text:  В Ингушетии возрождается древняя техника ковроваляния Ее представят на форуме-выставке «Россия» на ВДНХ.
Predicted: новость, статья, гибель иранского генерала касема сулеймани, конституция рф
-----------


 40%|███▉      | 441/1110 [02:39<04:01,  2.77it/s]

Batch: 440, Loss: 0.9786184430122375, Accuracy: 0.203125


 42%|████▏     | 470/1110 [02:50<03:52,  2.76it/s]


KeyboardInterrupt: 

In [51]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
model.eval()
classifier_model = KeywordClassifier(embedding_dim, number_keywords).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(classifier_model.parameters(), lr=1e-3)

epoch = 6
checkpoint = torch.load(f"/kaggle/working/checkpoint_{epoch}.pt")
classifier_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss_history = checkpoint['loss_history']

In [55]:
i = 0
row = full_dataset.data.iloc[i]
print("Text: ", row.Title, row.Description)
res = predict_keywords(row.Title, row.Description, "")
print("Predicted:", res)

Text:  Сериал на выходные: едкая детективная комедия с Розановой и Снигирь Рассказываем о густонаселенном сериале «Престиж», где каждый герой наделен смачным пороком и парой скелетов в шкафу
Predicted: ['дети 3-7 лет', 'наука и практика', 'опек', 'министерство финансов рф (минфин россии)', 'кндр (северная корея)', 'ирак', 'идлиб (мухафаза)', 'конституция рф', 'казахстан', 'беременность']


full_dataset.shape

In [144]:
for i in np.random.choice(full_dataset.data.shape[0], 10):
    row = full_dataset.data.iloc[i]
    print("Text: ", row.Title, row.Description)
    res = predict_keywords(row.Title, row.Description, "", top_keywords=10)
    print("Predicted:", ", ".join(res[4:]))
    print("-----------")

Text:  Полуметровый пенис разрушил жизнь своего обладателя Мужчина не может вступать в половые сношения, работать и найти подходящую одежду.
Predicted: религия и мировоззрение, протесты в белоруссии, биография, места, медицинский вестник, коронавирус в россии
-----------
Text:  Специалист рассказал, что портит вашу кожу Неправильное питание, недостаток спорта и курение могут состарить на десять лет.
Predicted: религия и мировоззрение, биография, медицинский вестник, протесты в белоруссии, места, коронавирус в россии
-----------
Text:  СМИ: американские военные застрелили в Сирии мирного жителя 
Predicted: религия и мировоззрение, протесты в белоруссии, биография, места, медицинский вестник, евросоюз
-----------
Text:  Вся правда о двухлетних детях: забавные фото Жить с двухлетним малышом – все равно что на вулкане: никогда не знаешь, что придет ребенку в голову через минуту. Зато именно в этом возрасте малыши способны на самые забавные выходки. Убедитесь в этом сами, посмотрев нашу под